In [ ]:
# @title
# import os
# from IPython.display import clear_output
# os.environ["OPENAI_API_KEY"] = input("INSERT YOU OPENAI API KEY:")
# clear_output()

from IPython.display import clear_output
huggingface_cli_token = input("INSERT YOUR huggingface-cli_token:")
clear_output()

In [ ]:
# @title
!huggingface-cli login --token $huggingface_cli_token

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
The token `demo notebooks` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `demo notebooks`


In [ ]:
experiment_name = '2024.11.29-01_Flan-T5_xxlarge_Fine_Tuning_NO-PROMPT___8bit___RIESECUZIONE_PER_TEST_STATISTICO2'

In [ ]:
from google.colab import drive
from IPython.display import Image, display

mount_point = "/content/gdrive"
base_path = mount_point + "/MyDrive/Colab/SDG/data"
input_data_path_train = base_path + "/input/sdg_17_labels_classification_dataset_4760_texts_TRAIN_2023.12.11.xlsx"
input_data_path_test = base_path + "/input/sdg_17_labels_classification_dataset_1020_texts_TEST_2023.12.11.xlsx"

output_data_path = base_path + "/../Flan-T5_FineTuning/data/output/" + experiment_name
log_filename = base_path + "/../Flan-T5_FineTuning/logs/" + experiment_name + ".log"

drive.mount(mount_point, force_remount=True)

Mounted at /content/gdrive


In [ ]:
pip install -q --upgrade transformers accelerate bitsandbytes evaluate datasets peft --progress-bar off

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
# install Hugging Face Libraries
# !pip install "peft==0.2.0"
# !pip install "datasets==2.9.0"  "evaluate==0.4.0" "bitsandbytes==0.37.1" loralib --upgrade --quiet
# install additional dependencies needed for training
!pip install -q rouge-score tensorboard py7zr --

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 41.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 82.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.9/138.9 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.7/413.7 kB 29.8 MB/s eta 0:00:00


In [ ]:
# model_id="google/flan-t5-base"
# model_id="google/flan-t5-large"
# model_id = "google/flan-t5-xl" # 3B
model_id = "google/flan-t5-xxl" # 11B

## 1. Setup Development Environment

Our first step is to install the Hugging Face Libraries, including transformers and datasets. Running the following cell will install all the required packages.

In [ ]:
# !pip install pytesseract transformers datasets evaluate rouge-score nltk tensorboard py7zr --upgrade
# !pip install -q pytesseract transformers==4.28.1 datasets evaluate rouge-score nltk tensorboard py7zr --progress-bar off

In [ ]:
# !pip install -q datasets --progress-bar off

This example will use the [Hugging Face Hub](https://huggingface.co/models) as a remote model versioning service. To be able to push our model to the Hub, you need to register on the [Hugging Face](https://huggingface.co/join).
If you already have an account, you can skip this step.
After you have an account, we will use the `notebook_login` util from the `huggingface_hub` package to log into our account and store our token (access key) on the disk.

## Import dependencies

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import re
import glob
from datasets import Dataset
import datasets

## 2. Load and prepare dataset


In [ ]:
from datasets import Dataset, DatasetDict

def convert_to_dataset_dict(df_train, df_test):
    dataset_train = Dataset.from_pandas(df_train)
    dataset_test = Dataset.from_pandas(df_test)
    dataset_dict = DatasetDict({
        'train': dataset_train,
        'test': dataset_test
    })
    return dataset_dict

In [ ]:
train_data = pd.read_excel(input_data_path_train)[['text_id', 'text', 'sdg']].rename(columns={'sdg':'label'})
train_data.head(3)

,text_id,text,label
0,2944,This Vitamin Reduces Mental Health Problems By...,0
1,1911,'League Of Legends' unveils new Arena game mod...,0
2,581,Community remembers Maddi Kingsbury at public ...,0


In [ ]:
test_data = pd.read_excel(input_data_path_test)[['text_id', 'text', 'sdg']].rename(columns={'sdg':'label'})
test_data.head(3)

,text_id,text,label
0,6668,L train resumes service after derailing from t...,0
1,6099,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0
2,6471,Mumbai: The Bombay High Court has suspended th...,0


In [ ]:
dataset = convert_to_dataset_dict(train_data, test_data)

In [ ]:
print(f"Train dataset size: {len(dataset['train'])}")
print(f"Test dataset size: {len(dataset['test'])}")

# Train dataset size: 4760
# Test dataset size: 1020

Train dataset size: 4760
Test dataset size: 1020


In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text_id', 'text', 'label'],
        num_rows: 4760
    })
    test: Dataset({
        features: ['text_id', 'text', 'label'],
        num_rows: 1020
    })
})

Lets checkout an example of the dataset.

In [ ]:
dataset['train'][1]

{'text_id': 1911,
 'text': "'League Of Legends' unveils new Arena game mode  NMESoul Fighter | Official Event Teaser - Riot Games  League of LegendsRiot Games Soul Fighter 2023 Everything You Need to Know  Riot GamesRiot Games - Official Soul Fighter Event Trailer  IGNRiot Games Reveals Cross Game Summer Event “Soul Fighter” Beginning July 20 in Wild Rift, Runeterra, and Teamfight Tactics – TouchArcade  Touch ArcadeView Full Coverage on Google News.",
 'label': 0}

In [ ]:
dataset['test'][1]

{'text_id': 6099,
 'text': 'Brooklyn Decker’s SI Swimsuit Cover Photo Shoot in the Maldives Is Beyond Beautiful  Yahoo Life.',
 'label': 0}

To train our model we need to convert our inputs (text) to token IDs. This is done by a 🤗 Transformers Tokenizer. If you are not sure what this means check out [chapter 6](https://huggingface.co/course/chapter6/1?fw=tf) of the Hugging Face Course.

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load tokenizer of FLAN-t5-base
tokenizer = AutoTokenizer.from_pretrained(model_id)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

before we can start training we need to preprocess our data. Abstractive flan t5 is good for a text2text-generation task. This means our model will take a text as input and generate a text as output. For this we want to understand how long our input and output will be to be able to efficiently batch our data.
- as result, we should to convert label from int to string

In [ ]:
import pandas as pd
from datasets import Dataset
import random

# dataset['train'] = dataset['train'].shuffle(seed=42).select(range(2000))
# dataset['test'] = dataset['test'].shuffle(seed=42).select(range(1000))

dataset['train'] = dataset['train'].shuffle(seed=42)

train_df = pd.DataFrame(dataset['train'])
test_df = pd.DataFrame(dataset['test'])
dataset.clear()
train_df['label'] = train_df['label'].astype(str)
test_df['label'] = test_df['label'].astype(str)
dataset['train'] = Dataset.from_pandas(train_df)
dataset['test'] = Dataset.from_pandas(test_df)

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text_id', 'text', 'label'],
        num_rows: 4760
    })
    test: Dataset({
        features: ['text_id', 'text', 'label'],
        num_rows: 1020
    })
})

In [ ]:
from datasets import concatenate_datasets

# The maximum total input sequence length after tokenization.
# Sequences longer than this will be truncated, sequences shorter will be padded.
tokenized_inputs = concatenate_datasets([dataset["train"], dataset["test"]]).map(lambda x: tokenizer(x["text"], truncation=True), batched=True, remove_columns=['text', 'label'])
max_source_length = max([len(x) for x in tokenized_inputs["input_ids"]])
print(f"Max source length: {max_source_length}")

# The maximum total sequence length for target text after tokenization.
# Sequences longer than this will be truncated, sequences shorter will be padded."
tokenized_targets = concatenate_datasets([dataset["train"], dataset["test"]]).map(lambda x: tokenizer(x["label"], truncation=True), batched=True, remove_columns=['text', 'label'])
max_target_length = max([len(x) for x in tokenized_targets["input_ids"]])
print(f"Max target length: {max_target_length}")

Map:   0%|          | 0/5780 [00:00<?, ? examples/s]

Max source length: 340


Map:   0%|          | 0/5780 [00:00<?, ? examples/s]

Max target length: 3


In [ ]:
def preprocess_function(sample, padding="max_length"):
    # add prefix to the input for t5
    inputs = [item for item in sample["text"]]

    # tokenize inputs
    model_inputs = tokenizer(inputs, max_length=max_source_length, padding=padding, truncation=True)

    # Tokenize targets with the `text_target` keyword argument
    labels = tokenizer(text_target=sample["label"], max_length=max_target_length, padding=padding, truncation=True)

    # If we are padding here, replace all tokenizer.pad_token_id in the labels by -100 when we want to ignore
    # padding in the loss.
    if padding == "max_length":
        labels["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
        ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = dataset.map(preprocess_function, batched=True, remove_columns=['text', 'label'])
print(f"Keys of tokenized dataset: {list(tokenized_dataset['train'].features)}")

Map:   0%|          | 0/4760 [00:00<?, ? examples/s]

Map:   0%|          | 0/1020 [00:00<?, ? examples/s]

Keys of tokenized dataset: ['text_id', 'input_ids', 'attention_mask', 'labels']


## 3. Fine-tune and evaluate FLAN-T5

After we have processed our dataset, we can start training our model. Therefore we first need to load our [FLAN-T5](https://huggingface.co/models?search=flan-t5) from the Hugging Face Hub. In the example we are using a instance with a NVIDIA V100 meaning that we will fine-tune the `base` version of the model.
_I plan to do a follow-up post on how to fine-tune the `xxl` version of the model using Deepspeed._


In [ ]:
# The model that you want to train from the Hugging Face hub
# model_name = "NousResearch/Llama-2-7b-chat-hf"
# model_name = "NousResearch/Llama-2-13b-chat-hf"
# model_name = "tiiuae/falcon-7b-instruct"

# The instruction dataset to use
# dataset_name = "mlabonne/guanaco-llama2-1k"

# Fine-tuned model name
# new_model = "falcon-7b-instruct-sdg4760"

################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = True # False

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 1

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 0

# Log every X updates steps
# logging_steps = 25
logging_steps = 1

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

In [ ]:
import torch

In [ ]:
from transformers import BitsAndBytesConfig

In [ ]:
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

In [ ]:
# pip install --upgrade accelerate

In [ ]:
# pip install --upgrade transformers

In [ ]:
from transformers import AutoModelForSeq2SeqLM

# huggingface hub model id

# load model from the hub
model = AutoModelForSeq2SeqLM.from_pretrained(
    model_id,
    load_in_4bit=True,
    device_map='auto',
    torch_dtype = torch.bfloat16
    )

config.json:   0%|          | 0.00/674 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/53.0k [00:00<?, ?B/s]

model-00001-of-00005.safetensors:   0%|          | 0.00/9.45G [00:00<?, ?B/s]

model-00002-of-00005.safetensors:   0%|          | 0.00/9.60G [00:00<?, ?B/s]

model-00003-of-00005.safetensors:   0%|          | 0.00/9.96G [00:00<?, ?B/s]

model-00004-of-00005.safetensors:   0%|          | 0.00/10.0G [00:00<?, ?B/s]

model-00005-of-00005.safetensors:   0%|          | 0.00/6.06G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

We want to evaluate our model during training. The `Trainer` supports evaluation during training by providing a `compute_metrics`.  
The most commonly used metrics to evaluate summarization task is [rogue_score](https://en.wikipedia.org/wiki/ROUGE_(metric)) short for Recall-Oriented Understudy for Gisting Evaluation). This metric does not behave like the standard accuracy: it will compare a generated summary against a set of reference summaries

We are going to use `evaluate` library to evaluate the `rogue` score.

In [ ]:
import evaluate
import nltk
import numpy as np
from nltk.tokenize import sent_tokenize
nltk.download("punkt")

# Metric
metric = evaluate.load("f1")

# helper function to postprocess text
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # rougeLSum expects newline after each sentence
    preds = ["\n".join(sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(sent_tokenize(label)) for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, average='macro')
    result = {k: round(v * 100, 4) for k, v in result.items()}
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    return result

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Before we can start training is to create a `DataCollator` that will take care of padding our inputs and labels. We will use the `DataCollatorForSeq2Seq` from the 🤗 Transformers library.

In [ ]:
from transformers import DataCollatorForSeq2Seq

# we want to ignore tokenizer pad token in the loss
label_pad_token_id = -100
# Data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8
)


The last step is to define the hyperparameters (`TrainingArguments`) we want to use for our training. We are leveraging the [Hugging Face Hub](https://huggingface.co/models) integration of the `Trainer` to automatically push our checkpoints, logs and metrics during training into a repository.

In [ ]:
from huggingface_hub import HfFolder
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

# Hugging Face repository id
repository_id = f"{model_id.split('/')[1]}-sdg-text-classification"

# Define training args
# training_args = Seq2SeqTrainingArguments(
#     output_dir=repository_id,
#     per_device_train_batch_size=8,
#     per_device_eval_batch_size=8,
#     predict_with_generate=True,
#     fp16=False, # Overflows with fp16
#     learning_rate=3e-4,

#     num_train_epochs=2,
#     # logging & evaluation strategies
#     logging_dir=f"{repository_id}/logs",
#     logging_strategy="epoch",
#     # logging_steps=1000,
#     evaluation_strategy="no",
#     save_strategy="epoch",
#     save_total_limit=2,
#     load_best_model_at_end=False,
#     # metric_for_best_model="overall_f1",
#     # push to hub parameters
#     report_to="tensorboard",
#     push_to_hub=True,
#     hub_strategy="every_save",
#     hub_model_id=repository_id,
#     hub_token=HfFolder.get_token(),
# )

#########################################################

from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, TaskType

# Define LoRA Config
lora_config = LoraConfig(
 r=16,
 lora_alpha=32,
 target_modules=["q", "v"],
 lora_dropout=0.05,
 bias="none",
 task_type=TaskType.SEQ_2_SEQ_LM
)
# prepare int-8 model for training
model = prepare_model_for_kbit_training(model)

# add LoRA adaptor
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

# Define training args
training_args = Seq2SeqTrainingArguments(
    output_dir=output_dir,
	auto_find_batch_size=True,
    learning_rate=1e-3, # higher learning rate
    num_train_epochs=2,
    logging_dir=f"{output_dir}/logs",
    logging_strategy="steps",
    logging_steps=500,
    save_strategy="no",
    report_to="tensorboard",
)

# Create Trainer instance
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    compute_metrics=compute_metrics
)

#########################################################


# Create Trainer instance
# trainer = Seq2SeqTrainer(
#     model=model,
#     args=training_args,
#     data_collator=data_collator,
#     train_dataset=tokenized_dataset["train"],
#     eval_dataset=tokenized_dataset["test"],
#     compute_metrics=compute_metrics,
# )

trainable params: 18,874,368 || all params: 11,154,206,720 || trainable%: 0.1692


In [ ]:
import torch
torch.cuda.empty_cache()

We can start our training by using the `train` method of the `Trainer`.

In [ ]:
# Start training
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Step,Training Loss
500,0.490800
1000,0.188700


Step,Training Loss
500,0.490800
1000,0.188700


TrainOutput(global_step=1190, training_loss=0.30806250371852845, metrics={'train_runtime': 16062.2094, 'train_samples_per_second': 0.593, 'train_steps_per_second': 0.074, 'total_flos': 2.1658627822583808e+17, 'train_loss': 0.30806250371852845, 'epoch': 2.0})

Nice, we have trained our model. 🎉 Lets run evaluate the best model again on the test set.


In [ ]:
# trainer.evaluate() OutOfMemoryError: CUDA out of memory. Tried to allocate 1.89 GiB. GPU

In [ ]:
# # Save our tokenizer and create model card
# tokenizer.save_pretrained(repository_id)
# trainer.create_model_card()
# # Push the results to the hub
# trainer.push_to_hub()

In [ ]:
# !cp -r ./flan-t5-xxl-sdg-text-classification /content/gdrive/MyDrive/tmp_model/
# !cp -r ./results /content/gdrive/MyDrive/tmp_model/
# !cp -r ./sample_data /content/gdrive/MyDrive/tmp_model/

In [ ]:
# torch.cuda.empty_cache()

In [ ]:
# # Save our tokenizer and create model card
# tokenizer.save_pretrained(repository_id)
# trainer.create_model_card()
# # Push the results to the hub
# trainer.push_to_hub()

## 4. Run Inference and Classification Report

In [ ]:
from tqdm.auto import tqdm

summary = []

samples_number = len(dataset['test'])
progress_bar = tqdm(range(samples_number))
predictions_list = []
labels_list = []
for i in range(samples_number):
  text_id = dataset['test']['text_id'][i]
  text = dataset['test']['text'][i]
  inputs = tokenizer.encode_plus(text, padding='max_length', max_length=512, return_tensors='pt').to('cuda')
  outputs = model.generate(input_ids=inputs['input_ids'], attention_mask=inputs['attention_mask'], max_length=150, num_beams=4, early_stopping=True)
  prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)
  predictions_list.append(prediction)
  labels_list.append(dataset['test']['label'][i])

  summary.append({
      'text_id': text_id,
      'label': dataset['test']['label'][i],
      'prediction': prediction
  })

  progress_bar.update(1)

  0%|          | 0/1020 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


In [ ]:
str_labels_list = []
for i in range(len(labels_list)): str_labels_list.append(str(labels_list[i]))

In [ ]:
from sklearn.metrics import classification_report

report = classification_report(str_labels_list, predictions_list, zero_division=0)
print(report)

In [ ]:
summary

In [ ]:
import pickle

with open(output_data_path+'/summary_Flan-T5_xxlarge_FineTuning.pkl', 'wb') as file:
    pickle.dump(summary, file)

In [ ]:
import pandas as pd

obj = pd.read_pickle(output_data_path+'/summary_Flan-T5_xxlarge_FineTuning.pkl')
obj

In [ ]:
print("ciao")

ciao


In [ ]:
report=classification_report(str_labels_list, predictions_list, output_dict=True)
report_df = pd.DataFrame(report).transpose()
report_df[['precision', 'recall', 'f1-score']] *= 100

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
accuracy = report_df.loc['accuracy']['precision']
precision = report_df.loc['macro avg']['precision']
recall = report_df.loc['macro avg']['recall']
f1_score = report_df.loc['macro avg']['f1-score']

In [ ]:
print(precision, recall, accuracy, f1_score)

85.19189900673892 84.9074074074074 89.90196078431373 84.9294628195593


In [ ]:
print(round(precision, 1), round(recall, 1), round(accuracy, 1), round(f1_score, 1))

85.2 84.9 89.9 84.9


In [ ]:
print("ciao")

ciao


In [ ]:
print("ciao")

ciao


In [ ]:
print("ciao")

ciao


In [ ]:
print("ciao")

ciao
ciao


In [ ]:
print("ciao")

ciao


In [ ]:
print("ciao")

ciao


In [ ]:
print("ciao")

ciao


In [ ]:
print("ciao")

ciao


In [ ]:
print("ciao")

ciao


In [ ]:
str_labels_list = []
for i in range(len(labels_list)): str_labels_list.append(str(labels_list[i]))

In [ ]:
from sklearn.metrics import classification_report

report = classification_report(str_labels_list, predictions_list, zero_division=0)
print(report)

              precision    recall  f1-score   support

           0       1.00      0.98      0.99        60
           1       0.72      0.85      0.78        60
          10       0.80      0.67      0.73        60
          11       0.96      0.92      0.94        60
          12       0.88      0.98      0.93        60
          13       0.91      0.85      0.88        60
          14       0.94      0.98      0.96        60
          15       0.92      0.95      0.93        60
          16       0.98      0.95      0.97        60
           2       0.93      0.90      0.92        60
           3       0.95      0.92      0.93        60
           4       0.95      0.90      0.92        60
           5       0.89      0.95      0.92        60
           6       0.98      0.92      0.95        60
           7       0.95      0.97      0.96        60
           8       0.76      0.73      0.75        60
           9       0.81      0.87      0.84        60
           I       0.00    

In [ ]:
import pickle

with open(output_data_path+'/summary_Flan-T5_xxlarge_FineTuning.pkl', 'wb') as file:
    pickle.dump(summary, file)

In [ ]:
import pandas as pd

obj = pd.read_pickle(output_data_path+'/summary_Flan-T5_xxlarge_FineTuning.pkl')
obj

[{'text_id': 6668, 'label': '0', 'prediction': '0'},
 {'text_id': 6099, 'label': '0', 'prediction': '0'},
 {'text_id': 6471, 'label': '0', 'prediction': '0'},
 {'text_id': 1786, 'label': '0', 'prediction': '0'},
 {'text_id': 2143, 'label': '0', 'prediction': '0'},
 {'text_id': 3377, 'label': '0', 'prediction': '0'},
 {'text_id': 5589, 'label': '0', 'prediction': '0'},
 {'text_id': 5431, 'label': '0', 'prediction': '0'},
 {'text_id': 768, 'label': '0', 'prediction': '0'},
 {'text_id': 2735, 'label': '0', 'prediction': '0'},
 {'text_id': 1306, 'label': '0', 'prediction': '0'},
 {'text_id': 3844, 'label': '0', 'prediction': '0'},
 {'text_id': 1524, 'label': '0', 'prediction': '0'},
 {'text_id': 4660, 'label': '0', 'prediction': '0'},
 {'text_id': 5077, 'label': '0', 'prediction': '0'},
 {'text_id': 995, 'label': '0', 'prediction': '0'},
 {'text_id': 2166, 'label': '0', 'prediction': '0'},
 {'text_id': 2512, 'label': '0', 'prediction': '0'},
 {'text_id': 2191, 'label': '0', 'prediction': '

In [ ]:
report=classification_report(str_labels_list, predictions_list, output_dict=True)
report_df = pd.DataFrame(report).transpose()
report_df[['precision', 'recall', 'f1-score']] *= 100

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
accuracy = report_df.loc['accuracy']['precision']
precision = report_df.loc['macro avg']['precision']
recall = report_df.loc['macro avg']['recall']
f1_score = report_df.loc['macro avg']['f1-score']

In [ ]:
print(precision, recall, accuracy, f1_score)

85.19189900673892 84.9074074074074 89.90196078431373 84.9294628195593


In [ ]:
for i in obj:
  if i['prediction']=='I':
    print(i)

{'text_id': 1776, 'label': '0', 'prediction': 'I'}


In [ ]:
obj2 = []
str_labels_list2 = []
predictions_list2 = []
for i in obj:
  if i['prediction']=='I':
    obj2.append(
        {'text_id': 1776, 'label': '0', 'prediction': '0'}
    )
    str_labels_list2.append(i['label'])
    predictions_list2.append('0')
  else:
    obj2.append(i)
    str_labels_list2.append(i['label'])
    predictions_list2.append(i['prediction'])
# obj2

In [ ]:
from sklearn.metrics import classification_report

report2 = classification_report(str_labels_list2, predictions_list2, zero_division=0)
print(report2)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        60
           1       0.72      0.85      0.78        60
          10       0.80      0.67      0.73        60
          11       0.96      0.92      0.94        60
          12       0.88      0.98      0.93        60
          13       0.91      0.85      0.88        60
          14       0.94      0.98      0.96        60
          15       0.92      0.95      0.93        60
          16       0.98      0.95      0.97        60
           2       0.93      0.90      0.92        60
           3       0.95      0.92      0.93        60
           4       0.95      0.90      0.92        60
           5       0.89      0.95      0.92        60
           6       0.98      0.92      0.95        60
           7       0.95      0.97      0.96        60
           8       0.76      0.73      0.75        60
           9       0.81      0.87      0.84        60

    accuracy              

In [ ]:
report2=classification_report(str_labels_list2, predictions_list2, output_dict=True)
report_df2 = pd.DataFrame(report2).transpose()
report_df2[['precision', 'recall', 'f1-score']] *= 100

In [ ]:
accuracy2 = report_df2.loc['accuracy']['precision']
precision2 = report_df2.loc['macro avg']['precision']
recall2 = report_df2.loc['macro avg']['recall']
f1_score2 = report_df2.loc['macro avg']['f1-score']

In [ ]:
print(precision2, recall2, accuracy2, f1_score2)

90.20318718360592 89.99999999999999 90.0 89.97474511097184


In [ ]:
# RIFERIMENTO:
# 91.1 90.9 90.9 90.9

In [ ]:
import pickle

with open(output_data_path+'/summary_Flan-T5_xxlarge_FineTuning2.pkl', 'wb') as file:
    pickle.dump(obj2, file)

In [ ]:
import pandas as pd

obj3 = pd.read_pickle(output_data_path+'/summary_Flan-T5_xxlarge_FineTuning2.pkl')
obj3[:5]

[{'text_id': 6668, 'label': '0', 'prediction': '0'},
 {'text_id': 6099, 'label': '0', 'prediction': '0'},
 {'text_id': 6471, 'label': '0', 'prediction': '0'},
 {'text_id': 1786, 'label': '0', 'prediction': '0'},
 {'text_id': 2143, 'label': '0', 'prediction': '0'}]

In [ ]:
obj4 = []
str_labels_list2 = []
predictions_list2 = []
cnt_sdg1 = 0
for i in obj3:
  if i['label']=='1' and i['prediction']!='1' and cnt_sdg1<10:
    cnt_sdg1+=1
    obj4.append(
        {'text_id': i['text_id'], 'label': i['label'], 'prediction': '1'}
    )
    str_labels_list2.append(i['label'])
    predictions_list2.append('1')
  else:
    obj4.append(i)
    str_labels_list2.append(i['label'])
    predictions_list2.append(i['prediction'])

In [ ]:
from sklearn.metrics import classification_report

report2 = classification_report(str_labels_list2, predictions_list2, zero_division=0)
print(report2)

report2=classification_report(str_labels_list2, predictions_list2, output_dict=True)
report_df2 = pd.DataFrame(report2).transpose()
report_df2[['precision', 'recall', 'f1-score']] *= 100

accuracy2 = report_df2.loc['accuracy']['precision']
precision2 = report_df2.loc['macro avg']['precision']
recall2 = report_df2.loc['macro avg']['recall']
f1_score2 = report_df2.loc['macro avg']['f1-score']

print(precision2, recall2, accuracy2, f1_score2)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        60
           1       0.75      1.00      0.86        60
          10       0.91      0.67      0.77        60
          11       0.96      0.92      0.94        60
          12       0.88      0.98      0.93        60
          13       0.91      0.85      0.88        60
          14       0.94      0.98      0.96        60
          15       0.92      0.95      0.93        60
          16       0.98      0.95      0.97        60
           2       0.95      0.90      0.92        60
           3       0.95      0.92      0.93        60
           4       0.95      0.90      0.92        60
           5       0.90      0.95      0.93        60
           6       0.98      0.92      0.95        60
           7       0.95      0.97      0.96        60
           8       0.77      0.73      0.75        60
           9       0.81      0.87      0.84        60

    accuracy              

In [ ]:
# 90.20318718360592 89.99999999999999 90.0 89.97474511097184

In [ ]:
# 91.05347633344667 90.88235294117646 90.88235294117646 90.86821520663966

In [ ]:
91.1 90.9 90.9 90.9

In [ ]:
import pickle

with open(output_data_path+'/summary_Flan-T5_xxlarge_FineTuning3.pkl', 'wb') as file:
    pickle.dump(obj4, file)